<a href="https://colab.research.google.com/github/detektor777/colab_list_image/blob/main/sdxl_1_0_lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title ##**Install** { display-mode: "form" }
%%capture

!pip install -q invisible_watermark transformers accelerate safetensors diffusers xformers mediapy
!pip install gdown
import torch
from diffusers import StableDiffusionXLImg2ImgPipeline, DiffusionPipeline, KDPM2AncestralDiscreteScheduler, StableDiffusionXLPipeline, AutoencoderKL
import gc
from PIL import Image
import requests
from io import BytesIO
from IPython.display import display
import os
import random
import re
import gdown
from IPython.display import display, Image
from IPython.display import clear_output
import ipywidgets as widgets

model_base = "stabilityai/stable-diffusion-xl-base-1.0"
v_autoencoder = "madebyollin/sdxl-vae-fp16-fix" # fix vae for run in fp16 precision without generating NaNs

vae = AutoencoderKL.from_pretrained(v_autoencoder, torch_dtype=torch.float16)

pipe = StableDiffusionXLPipeline.from_pretrained(
    model_base,
    vae=vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
    add_watermarker=False, # no watermarker
    )

pipe.safety_checker = None

pipe.to("cuda")
model_refiner = "stabilityai/stable-diffusion-xl-refiner-1.0"

pipe_refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    model_refiner,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
    add_watermarker=False, # no watermarker
    )

#pipe_refiner.to("cuda")
pipe_refiner.enable_model_cpu_offload()
#(Optional) Change the scheduler

#pipe.scheduler = KDPM2AncestralDiscreteScheduler.from_config(
 # pipe.scheduler.config, use_karras_sigmas=False
#)

#generator = torch.Generator().manual_seed(42)

from diffusers.schedulers.scheduling_dpmsolver_multistep import DPMSolverMultistepScheduler
from diffusers import StableDiffusionXLPipeline, StableDiffusionPipeline
from diffusers import DPMSolverMultistepScheduler, EulerDiscreteScheduler
from diffusers import KDPM2DiscreteScheduler, KDPM2AncestralDiscreteScheduler
from diffusers import EulerAncestralDiscreteScheduler, HeunDiscreteScheduler
from diffusers import LMSDiscreteScheduler


In [ ]:
#@title ##**Load lora model (Optional)**  { display-mode: "form" }
#@markdown Load a model from a link or in this folder '/content'
#%%capture

lora_name = '' #@param {type:"string"}
model_link = "" #@param {type:"string"}

if not lora_name or not model_link:
    print("lora_name or model_link is empty")
else:
    model_name = f"{lora_name}.safetensors"
    url = model_link

    if "drive.google.com" in url:
        file_id_match = re.search(r'file/d/(.*?)/', url)
        file_id = file_id_match.group(1)
        print(f"file_id: {file_id}")
        url = f'https://drive.google.com/uc?id={file_id}'
        destination_path = f'/content/{lora_name}.safetensors'
        gdown.download(url, destination_path, quiet=False)
    else:
        print(f"link")
        !wget "{url}" -O "{model_name}"

    clear_output(wait=True)

In [ ]:
#@title ##**Select lora model (Optional)**  { display-mode: "form" }

folder_path = '/content'

files = [f for f in os.listdir(folder_path) if f.endswith('.safetensors')]

if not files:
    print("not lora models")
else:
    files_str = ', '.join(files)
    print(f"Lora models: {files_str}")

    selected_files = widgets.Dropdown(
        options=files,
        value=files[0],
        description='Select:'
    )

    selected_file_value = selected_files.value

    def on_value_change(change):
        global selected_file_value
        selected_file_value = change['new']
        print(f"Selected: {selected_file_value}")

    selected_files.observe(on_value_change, names='value')

    display(selected_files)


In [ ]:
#@title ##**Create and show** { display-mode: "form" }

lora_status = "Yes" #@param ["Yes","No"]
lora_scale = "1.0" #@param ["0.0", "0.1", "0.2", "0.3", "0.4", "0.5", "0.6", "0.7", "0.8", "0.9", "1.0","1.5","2.0"]
lora_scale = float(lora_scale)

if 'selected_file_value' in locals() and selected_file_value and lora_status == "Yes":
    pipe.load_lora_weights(selected_file_value)
    pipe.fuse_lora(lora_scale=lora_scale)
    print(f"Selected: {selected_file_value}")
else:
    pipe.unfuse_lora()
    pipe.unload_lora_weights()
    print("Lora is empty.")

prompt =  "cyberpunk, cat, Ukrainian flag" #@param {type:"string"}
prompt2 = ""

negative_prompt = ''#@param {type:"string"}
negative_prompt2 = ''

scale = 10 #@param {type:"integer"}
steps = 50 #@param {type:"integer"}

height = 1024 #@param {type:"integer"}
width = 1024 #@param {type:"integer"}

scheduler_name = "LMS" #@param ["Euler_a","Euler", "Euler_K", "LMS", "LMS_Karras", "Heun", "DPMPP_2M", "DPMPP_2M_Karras", "DPMPP_2M_Lu", "DPMPP_2M_Stable", "DPMPP_2M_SDE", "DPMPP_2M_SDE_Karras", "DPMPP_2M_SDE_Lu", "DPMPP_2M_SDE_Stable", "DPM++_2M_Karras", "DPM_Fast"]

schedulers = {
    "Euler_K": (EulerDiscreteScheduler, {"use_karras_sigmas": True}),

    "DPMPP_2M": (DPMSolverMultistepScheduler, {}),
    "DPMPP_2M_Karras": (DPMSolverMultistepScheduler, {"use_karras_sigmas": True}),
    "DPMPP_2M_Lu": (DPMSolverMultistepScheduler, {"use_lu_lambdas": True}),
    "DPMPP_2M_Stable": (DPMSolverMultistepScheduler, {"euler_at_final": True}),

    "DPMPP_2M_SDE": (DPMSolverMultistepScheduler, {"algorithm_type": "sde-dpmsolver++"}),
    "DPMPP_2M_SDE_Karras": (DPMSolverMultistepScheduler, {"use_karras_sigmas": True, "algorithm_type": "sde-dpmsolver++"}),
    "DPMPP_2M_SDE_Lu": (DPMSolverMultistepScheduler, {"use_lu_lambdas": True, "algorithm_type": "sde-dpmsolver++"}),
    "DPMPP_2M_SDE_Stable": (DPMSolverMultistepScheduler, {"algorithm_type": "sde-dpmsolver++", "euler_at_final": True}),

    "DPM++_2M_Karras": (DPMSolverMultistepScheduler, {"use_karras_sigmas": True, "algorithm_type": "sde-dpmsolver++", "steps": 70}),
    "DPM_Fast": (DPMSolverMultistepScheduler, {"algorithm_type": "sde-dpmsolver++", "steps": 100}),

    "DPM2": (KDPM2DiscreteScheduler, {}),
    "DPM2_Karras": (KDPM2DiscreteScheduler, {"use_karras_sigmas": True}),
    "DPM2_a": (KDPM2AncestralDiscreteScheduler, {}),
    "DPM2_a_Karras": (KDPM2AncestralDiscreteScheduler, {"use_karras_sigmas": True}),

    "Euler": (EulerDiscreteScheduler, {'beta_start': 0.0006, 'beta_end': 0.012, 'beta_schedule': "scaled_linear", 'steps_offset': 0}),
    "Euler_a": (EulerAncestralDiscreteScheduler, {}),

    "Heun": (HeunDiscreteScheduler, {}),

    "LMS": (LMSDiscreteScheduler, {}),
    "LMS_Karras": (LMSDiscreteScheduler, {"use_karras_sigmas": True}),
}

#'beta_start': 0.0006, 'beta_end': 0.012, 'beta_schedule': "scaled_linear", 'steps_offset': 0
model_id = model_base

params = {
    "prompt":prompt,
    "prompt_2":prompt2,
    "height":height,
    "width":width,
    "negative_prompt":negative_prompt,
    "negative_prompt_2":negative_prompt2,
    "guidance_scale":scale,
    "num_inference_steps":steps
}

save_dir = './samples_sdxl'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

seed = random.randint(0, 1000000000000000000)
seed = 333
generator = torch.Generator(device='cuda').manual_seed(seed)

scheduler = DPMSolverMultistepScheduler.from_pretrained(
    model_id,
    subfolder="scheduler",
    **schedulers[scheduler_name][1],
)
pipe.scheduler = scheduler
clear_output(wait=True)

sdxl_img = pipe(**params, generator=generator).images[0]

sdxl_img.save(os.path.join(save_dir, f"seed_{seed}_steps_{params['num_inference_steps']}_{scheduler_name}.png"))

# Unload lora
pipe.unfuse_lora()
pipe.unload_lora_weights()

gc.collect()
torch.cuda.empty_cache()
torch.cuda.synchronize()

#sdxl_img

image_path = os.path.join(save_dir, f"seed_{seed}_steps_{params['num_inference_steps']}_{scheduler_name}.png")

if os.path.isfile(image_path):
    clear_output(wait=True)
    display(Image(filename=image_path, width=700))
else:
    print(f"File '{image_path}' not found")


In [ ]:
#@title ##**Refine and show (optional)** { display-mode: "form" }
#@markdown running this code might lead to insufficient memory
gc.collect()
torch.cuda.empty_cache()
torch.cuda.synchronize()

denoising_start = "0.85" #@param [0.7, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99]
denoising_start = float(denoising_start)

steps = 50 #@param {type:"integer"}

image_refined = pipe_refiner(prompt=prompt, num_inference_steps=steps, denoising_start=denoising_start, image=sdxl_img).images[0]

gc.collect()
torch.cuda.empty_cache()
torch.cuda.synchronize()

image_path = os.path.join(save_dir, f"seed_{seed}_steps_{params['num_inference_steps']}_{scheduler_name}engance.png")

image_refined.save(os.path.join(save_dir, f"seed_{seed}_steps_{params['num_inference_steps']}_{scheduler_name}engance.png"))


if os.path.isfile(image_path):
    clear_output(wait=True)
    display(Image(filename=image_path, width=700))
else:
    print(f"File '{image_path}' not found")
